In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
training_set_metadata = pd.read_csv("all/training_set_metadata.csv")
training_set_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [4]:
training_set = pd.read_csv("all/training_set.csv")
training_set.describe()

,object_id,mjd,passband,flux,flux_err,detected
count,1.421705e+06,1.421705e+06,1.421705e+06,1.421705e+06,1.421705e+06,1.421705e+06
mean,3.392608e+07,6.017921e+04,2.783108e+00,2.468855e+01,2.548932e+01,1.666168e-01
std,4.258439e+07,3.092379e+02,1.703946e+00,3.684506e+03,3.814492e+03,3.726336e-01
min,6.150000e+02,5.958003e+04,0.000000e+00,-1.149388e+06,4.637530e-01,0.000000e+00
25%,1.843400e+05,5.989905e+04,1.000000e+00,-2.789418e+00,2.118670e+00,0.000000e+00
50%,4.548783e+06,6.019331e+04,3.000000e+00,7.331990e-01,4.708105e+00,0.000000e+00
75%,6.796280e+07,6.048722e+04,4.000000e+00,8.830932e+00,1.297371e+01,0.000000e+00
max,1.307798e+08,6.067436e+04,5.000000e+00,2.432809e+06,2.234069e+06,1.000000e+00


In [6]:
targets = training_set_metadata.target.unique()
n_classes = targets.size # number of known classes
n_classes

14

In [2]:
class TWED:
    def __init__(self, lamda, stiffness, p = 1):
        self.lamda = lamda
        self.stiffness = stiffness
        self.p = p
    
    def lp_norm(self, a, b):
        c = np.abs(a - b)
        return np.power(c, self.p).sum() ** (1/self.p)
    
    def similarity(self, A, B, time_stamp_A, time_stamp_B):
        seq_A = np.append([0], A)
        seq_B = np.append([0], B)
        ts_A = np.append([0], time_stamp_A)
        ts_B = np.append([0], time_stamp_B)
        n = len(ts_A)
        m = len(ts_B)
        twed = np.full((n, m), np.inf)
        twed[0, 0] = 0
        for i in range(1, n):
            for j in range(1, m):
                del_A = (twed[i-1, j] + 
                         self.lp_norm(seq_A[i-1], seq_A[i]) + 
                         self.stiffness * self.lp_norm(ts_A[i], ts_A[i-1]) + 
                         self.lamda)
                del_B = (twed[i, j-1] + 
                         self.lp_norm(seq_B[j], seq_B[j-1]) + 
                         self.stiffness * self.lp_norm(ts_B[j], ts_B[j-1]) + 
                         self.lamda)
                match = (twed[i-1, j-1] +
                         self.lp_norm(seq_A[i], seq_B[j]) +
                         self.stiffness * self.lp_norm(ts_A[i], ts_B[j]) + 
                         self.lp_norm(seq_A[i-1], seq_B[j-1]) + 
                         self.stiffness * self.lp_norm(ts_A[i-1], ts_B[j-1]))
                
                twed[i, j] = np.min([del_A, del_B, match])
        return twed[n-1, m-1]

In [64]:
def preprocessing(aDataFrame):
    object_ids = aDataFrame.object_id.unique()
    df = pd.concat([aDataFrame[aDataFrame.object_id == oid] for oid in object_ids], ignore_index=True)
    flux_mean = df.flux.mean()
    flux_std = df.flux.std()
    normalized_df = df.copy()
    normalized_df["flux"] = df.flux / df.flux_err
    normalized_df["flux"] = (df.flux - flux_mean) / flux_std
    return normalized_df

In [65]:
normalized_df = preprocessing(training_set)

In [69]:
sample_targets = targets[:4]
oids = pd.concat(
    [training_set_metadata[training_set_metadata.target==t][["object_id", "target"]][:5] for t in sample_targets]
)
sample_data = pd.concat([normalized_df[normalized_df.object_id == oid] for oid in oids.object_id], ignore_index=True)
sample_data.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-0.154566,3.622952,1
1,615,59750.4306,1,-0.228286,5.553370,1
2,615,59750.4383,3,-0.134638,3.801213,1
3,615,59750.4450,4,-0.112274,11.395031,1
4,615,59752.4070,2,-0.191762,4.041204,1


In [73]:
oids.head()

,object_id,target
0,615,92
69,12695,92
150,26161,92
156,26783,92
162,28391,92


In [50]:
twed = TWED(lamda=1, stiffness=1)

In [78]:
def twed_to_a_class(twed, aSeries, class_df):
    oids = class_df.object_id.unique()
    tweds = np.array([twed.similarity(
                aSeries, 
                class_df[class_df==oid].flux.values, 
                aSeries.index, 
                class_df[class_df==oid].mjd.values) for oid in oids])
    return tweds.mean()

In [80]:
aSeries = normalized_df[normalized_df.object_id==615].object_id
aSeries.index = normalized_df[normalized_df.object_id==615].mjd.values
oid_in_class = oids[oids.target==92].object_id.values[:5]
class_df = pd.concat([normalized_df[normalized_df.object_id==i] for i in oid_in_class])
twed_to_a_class(twed, aSeries, class_df)

nan